### sử dụng với dữ liệu creditcard https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download

In [46]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


In [47]:
df = pd.read_csv('Train.csv')
df.replace('?', pd.NA, inplace=True)

# Fill missing values for categorical columns with the mode (most common value)
for column in df.select_dtypes(include=['object']).columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

# Fill missing values for numerical columns (if any) with the mean or median
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    df[column].fillna(df[column].mean(), inplace=True)

# Convert categorical variables to numerical using one-hot encoding
df = pd.get_dummies(df, drop_first=True)

In [48]:
def create_datastream(data, batch_size):
    datastream = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size]
        datastream.append(batch)
    return datastream
batch_size = 150
S = create_datastream(df, batch_size)

In [49]:
def train_random_forest_classifier(X, y, n_estimators=20):
    """
    Huấn luyện bộ phân loại Random forest trên dữ liệu X, y.
    n_estimators: Số lượng cây trong Random forest.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf

In [50]:
def adaptive_ensemble_size(C, sample, unique_label, anpha, min_num = 3):
    """
    Cơ chế adaptive_esemble 
    C : Bộ phân loại yếu
    sample
    unique_label : Tập hợp nhãn trong luồng dữ liệu
    anpha : hyper parameter
    min_num : số lượng C tối thiểu 
    """
    probability_list = []
    for i in range(len(unique_label)):
        li = [] 
        for j in range(len(C)):
            probabilities = C[0].predict_proba(sample)[0]
            probability_dict = {label: prob for label, prob in zip(C[0].classes_, probabilities)}
            latest_proba = probability_dict.get(unique_label[i], 0)
            probabilities1 = C[j].predict_proba(sample)[0]
            probability_dict1 = {label: prob for label, prob in zip(C[j].classes_, probabilities1)}
            current_proba = probability_dict1.get(unique_label[i], 0)
            if len(li) < min_num:
                li.append(current_proba)
            else:
                if abs(current_proba - latest_proba) < anpha :
                    li.append(current_proba)
                else: 
                    break   
        probability_list.append(li)     
    return probability_list

In [51]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    for i in range(len(Y)):
        li = probability_list[i]
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        slope, intercept = linear_regression(x, y)
        next_value = slope * (len(li) + 1) + intercept
        li.append(next_value)
        weighted_prob = sum([li[x - 1] * (1 + x * epsilon) for x in range(1, len(li) + 1)]) / len(li)
        predicted_probabilities.append(weighted_prob)
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps


In [52]:

def process_data_stream(S, m, k, unique_labels):
    """
    Processes data stream S using a weak Random Forest classifier.
    - S: Data stream (list of data blocks Bi)
    - m: Max size of classifier set C
    - k: Min size of C to make predictions
    """
    C = deque(maxlen=m) 
    pre = []
    for i in range(len(S) - 1):
        Bi = S[i]
        block_predictions = []
        X = Bi.iloc[:, :-1] 
        y = Bi.iloc[:, -1]   
        Ci = train_random_forest_classifier(X, y)
        C.append(Ci)
        if len(C) < k:
            continue
        Bi_1 = S[i + 1]
        for index, row in Bi_1.iterrows():
            sample = pd.DataFrame([row[:-1]], columns=Bi.columns[:-1])
            anpha = (30 / len(Bi_1)) * 0.2
            selected_classifiers = adaptive_ensemble_size(C, sample, unique_labels, anpha)
            pre_sample = tendency_prediction(selected_classifiers, unique_labels)
            block_predictions.append(pre_sample)
        pre.append(block_predictions)
    return C, pre
m = 15
k = 3
unique_labels = set(df.iloc[:, -1])
unique_labels = list(unique_labels)
C, pre = process_data_stream(S, m, k, unique_labels)


In [53]:
# Giả sử rằng pre là danh sách chứa các dự đoán và bạn muốn so sánh với các nhãn thực tế từ Bi_1
y_true = []
y_pred = []

for i in range(2, len(pre)):
    # Thêm nhãn thực tế từ Bi_1
    Bi_1 = S[i + 1]  # Chúng ta đã sử dụng Bi_1 trong process_data_stream
    y_true.extend(Bi_1.iloc[:, -1].tolist())  # Nhãn thực tế
    
    # Thêm dự đoán vào danh sách y_pred
    y_pred.extend(pre[i])  # Dự đoán từ từng block

# Chuyển đổi về dạng numpy array nếu cần
y_true = np.array(y_true)
y_pred = np.array(y_pred)
f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1}")

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

F1 Score: 0.0
Accuracy: nan%


c:\Users\MyPC\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\MyPC\AppData\Roaming\Python\Python39\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\MyPC\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
